In [246]:
from openai import OpenAI
from secret import OPENAI_API_KEY
from PIL import Image
import base64
import os
import json
import re
import pandas as pd
import io

In [247]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-4o-mini"

In [248]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [249]:
def old_encode_image(image_path):
     try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

     except Exception as e:

        print(f"Erreur lors de l'encodage de l'image {image_path} : {e}")
        return None

In [250]:
def send_chat_request(message):
    try:
        response = client.chat.completions.create(
            model=model, messages=message
        )

        result = response.choices[0].message.content.strip()
        result = extract_json(result)

        tokens = response.usage.total_tokens

        return result, tokens


    except Exception as e:
        print(f"Erreur OpenAI : {e}")
        return -1, None

In [251]:
def chat_get_key_words(image_paths):

    # Liste pour chaque image et chaque texte associé
    content_list = []
    for image_path in image_paths:
        print(f"Image : {image_path}")
        base64_image = encode_image(image_path)
        image_name = os.path.basename(image_path)
        content_list.append({
            "type": "text",
            "text": f"""Décris moi l'image avec 5 mots-clés.Retourne le résultat au format JSON : {{ {image_name} : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5] }} """
        })
        content_list.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        })

    messages = [
        {
            "role": "user",
            "content": content_list
        }
    ]

    return send_chat_request(messages)


In [252]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [253]:
def chat_get_categories(keywords_output):
    """
    Utilise les mots-clés extraits pour regrouper les images similaires en catégories.
    Les images sont identifiées par leur ordre dans la liste.
    """

    # Préparation d'un prompt détaillé incluant le résultat des mots-clés et l'ordre des images
    prompt = f"""Voici les listes de mots-clés obtenues pour chaque image (dans l'ordre) : {keywords_output}
    Regroupe les images similaires dans des catégories. Une catégorie est décrite par un seul mot-clé. Une image ne peut appartenir qu'à une seule catégorie. Retourne le résultat au format JSON : {{ "categorie1": [ "name", "name" ], "categorie2": [ "name", "name" ],...}}"""

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    return send_chat_request(messages)

In [254]:
directory = "test_data"
allowed_extensions = {".jpg", ".jpeg", ".png"}
image_paths = [
    os.path.join(directory, filename)
    for filename in os.listdir(directory)
    if os.path.splitext(filename)[1].lower() in allowed_extensions
]

In [255]:
keywords_output, tokens_keywords = chat_get_key_words(image_paths)
print("Mots-clés par image :")
print(keywords_output)
print(tokens_keywords)

Image : test_data\20240822_142034.jpg
Image : test_data\20240822_142036.jpg
Image : test_data\20240822_142048.jpg
Image : test_data\20240825_121144.jpg
Image : test_data\20240826_184159.jpg
Image : test_data\20240827_084810.jpg
Image : test_data\20240827_084914.jpg
Image : test_data\20240827_092136.jpg
Image : test_data\20240828_141857.jpg
Image : test_data\20240828_174310.jpg
Image : test_data\20240828_211839.jpg
Image : test_data\20240830_104749.jpg
Image : test_data\20240831_160656.jpg
Image : test_data\20240831_161613.jpg
Image : test_data\20240831_180654.jpg
Image : test_data\20240831_194108.jpg
Image : test_data\20240902_150137.jpg
Image : test_data\20240902_152654.jpg
Image : test_data\20240902_162507.jpg
Image : test_data\20250213_165625.jpg
Image : test_data\20250217_081138.jpg
Image : test_data\20250219_084504.jpg
Mots-clés par image :
{'20240822_142034.jpg': ['bâtiment', 'rue', 'arbres', 'nuage', 'voiture'], '20240822_142036.jpg': ['bâtiment', 'fenêtres', 'rue', 'arbres', 'm

In [256]:
categories_output, tokens_categories = chat_get_categories(keywords_output)
print("Catégorisation des images :")
print(categories_output)
print(tokens_categories)

Catégorisation des images :
{'bâtiments': ['20240822_142034.jpg', '20240822_142036.jpg', '20240822_142048.jpg', '20240827_084810.jpg', '20240827_092136.jpg'], 'snacks': ['20240828_141857.jpg'], 'courrier': ['20240828_174310.jpg'], 'repas': ['20240828_211839.jpg', '20240831_194108.jpg'], 'étiquettes': ['20240830_104749.jpg'], 'magasin': ['20240831_160656.jpg', '20240831_161613.jpg'], 'thé': ['20240831_180654.jpg'], 'nature': ['20240826_184159.jpg', '20240902_150137.jpg', '20240902_152654.jpg', '20240902_162507.jpg'], 'hiver': ['20250213_165625.jpg', '20250217_081138.jpg', '20250219_084504.jpg'], 'événements': ['20240827_084914.jpg']}
1075


In [257]:
total_tokens = tokens_keywords + tokens_categories
total_tokens

7901

## Ajout des données au DataFrame

In [258]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])

    return df

In [259]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        image_data["Keywords"] = image_data["image_name"].apply(lambda img: keywords_output.get(img, None))
    return image_data

In [260]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        image_data["Category"] = image_data["image_name"].map(lambda image: next((cat for cat in categories_output if image in categories_output[cat]), None))
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [261]:
image_data = create_df(image_paths)

image_data

,image_name,path,date_time,localisation
0,20240822_142034.jpg,test_data\20240822_142034.jpg,2024:08:22 14:20:34,None
1,20240822_142036.jpg,test_data\20240822_142036.jpg,2024:08:22 14:20:37,None
2,20240822_142048.jpg,test_data\20240822_142048.jpg,2024:08:22 14:20:48,None
3,20240825_121144.jpg,test_data\20240825_121144.jpg,2024:08:25 12:11:44,None
4,20240826_184159.jpg,test_data\20240826_184159.jpg,2024:08:26 18:41:59,None
5,20240827_084810.jpg,test_data\20240827_084810.jpg,2024:08:27 08:48:10,None
6,20240827_084914.jpg,test_data\20240827_084914.jpg,2024:08:27 08:49:14,None
7,20240827_092136.jpg,test_data\20240827_092136.jpg,2024:08:27 09:21:36,None
8,20240828_141857.jpg,test_data\20240828_141857.jpg,2024:08:28 14:18:58,None
9,20240828_174310.jpg,test_data\20240828_174310.jpg,2024:08:28 17:43:11,None


In [262]:
image_data = add_keywords_to_df(image_data, keywords_output)
image_data = add_categories_to_df(image_data, categories_output)

In [263]:
image_data

,image_name,path,date_time,localisation,Keywords,Category
0,20240822_142034.jpg,test_data\20240822_142034.jpg,2024:08:22 14:20:34,None,"[bâtiment, rue, arbres, nuage, voiture]",bâtiments
1,20240822_142036.jpg,test_data\20240822_142036.jpg,2024:08:22 14:20:37,None,"[bâtiment, fenêtres, rue, arbres, modernité]",bâtiments
2,20240822_142048.jpg,test_data\20240822_142048.jpg,2024:08:22 14:20:48,None,"[bâtiment, architecture, voitures, rue, croise...",bâtiments
3,20240825_121144.jpg,test_data\20240825_121144.jpg,2024:08:25 12:11:44,None,"[beurre, cacahuète, pot, oursons, produit]",None
4,20240826_184159.jpg,test_data\20240826_184159.jpg,2024:08:26 18:41:59,None,"[sentier, nature, arbres, feuilles, chemin]",nature
5,20240827_084810.jpg,test_data\20240827_084810.jpg,2024:08:27 08:48:10,None,"[bâtiment, espace, pelouse, structure, extérieur]",bâtiments
6,20240827_084914.jpg,test_data\20240827_084914.jpg,2024:08:27 08:49:14,None,"[événement, stands, coin, participants, tente]",événements
7,20240827_092136.jpg,test_data\20240827_092136.jpg,2024:08:27 09:21:36,None,"[bâtiment, drapeaux, événement, espace, extéri...",bâtiments
8,20240828_141857.jpg,test_data\20240828_141857.jpg,2024:08:28 14:18:58,None,"[snack, pretzel, produit, emballage, doux]",snacks
9,20240828_174310.jpg,test_data\20240828_174310.jpg,2024:08:28 17:43:11,None,"[boîte, courrier, boîtes aux lettres, extérieu...",courrier
